# Tarea 4
En esta tarea usted deberá completar el siguiente notebook, en el cual va a implementar el protocolo de firmas de anillo sobre grupos en general. Debe completar exclusivamente las partes marcadas con `##### POR COMPLETAR`.

## Funciones auxiliares
Necesitaremos un test de primalidad para revisar que no nos estén engañando cuando nos entreguen los parámetros del grupo.

In [1]:
import random

def _is_natural_power(n):
    # Para cada posible exponente, hacemos búsqueda binaria de la base
    search_exponent = 2
    
    # Optimiazación: si n no es a ^ k no puede ser a ^ (kr) para ningún
    # r, por lo que sólo probamos con exponentes primos
    avoid_exponents = set()
    
    while pow(2, search_exponent) <= n:
        
        if search_exponent not in avoid_exponents:
            # Usamos búsqueda binaria "creciente" para definir el intervalo
            # inicial
            search_start = 2
            i = 2
            while search_start ** search_exponent < n:
                search_start *= 2
                avoid_exponents.add(search_exponent * i)
                i += 1
                
            upper = search_start
            lower = search_start // 2

            # Búsqueda binaria
            while lower != upper:
                mid = (upper + lower) // 2
                result = pow(mid, search_exponent)
                if result < n:
                    lower = mid + 1
                elif result > n:
                    upper = mid
                else:
                    return True

            # Caso borde en que upper ^ search_exponent era justo n
            if pow(upper, search_exponent) == n:
                return True
            
        search_exponent += 1
    
    return False

In [2]:
def _extended_euclid(a, b):
    if a > b:
        return _extended_euclid_base(a, b)
    return _extended_euclid_base(b, a)

def _extended_euclid_base(a, b):
    prev_r, r = a, b
    prev_s, s = 1, 0
    prev_t, t = 0, 1

    while r != 0:
        q = prev_r // r
        prev_r, r = r, prev_r % r
        prev_s, s = s, prev_s - q * s
        prev_t, t = t, prev_t - q * t

    return prev_r, prev_s, prev_t

In [3]:
def is_probably_prime(n, iterations=100):
    if n == 2:
        return True
    if n % 2 == 0 or n == 1:
        return False
    if _is_natural_power(n):
        return False
    
    found_negative = False
    for i in range(iterations):
        a = random.randint(1, n - 1)
        if _extended_euclid(a, n)[0] > 1:
            return False
        b = pow(a, (n - 1) // 2, n)
        if b == n - 1:
            found_negative = True
        elif b != 1:
            return False
    
    return found_negative

## Un primer grupo
Como un primer ejemplo de grupo, consideramos a los grupos Z<sub>p</sub><sup>\*</sup> vistos en clases. En particular, definimos la clase `ZpStar` cuyas instancias son estos grupos. Para representar a los elementos dentro de Z<sub>p</sub><sup>\*</sup>, en su constructor se crea dinámicamente la clase `Element`.

In [4]:
class ZpStar:
    
    def __init__(self, p):
        if not is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        class Element:
            def __init__(self, value):
                if value < 1 or value > p-1:
                    raise Exception(f"value={value} is not in the range 1,...,{p-1}")
                self.value = value

            # Allows to compare elements with ==
            def __eq__(self, other_element):
                return self.value == other_element.value

            # Allows to operate elements with *
            def __mul__(self, other_element):
                return Element((self.value * other_element.value) % p)

            # Allows to use ** as exponentiation
            def __pow__(self, exponent):
                return Element(pow(self.value, exponent, p))

            # Allows to use str(e) to transform an element into a string
            def __str__(self):
                return str(self.value)

        self.element_class = Element
                
    def get_identity(self):
        return self.get_element(1)
    
    def get_element(self, n):
        return self.element_class(n)

Ahora vamos a definir una clase que inicializa a los participantes del anillo. Cada participante es una instancia de la clase `Signer` que será definida más adelante. De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `RingSetup` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `subgroup_order` no es un número primo, entonces se debe generar una excepción (puede suponer que el valor entregado en `subgroup_order` va a ser un número natural mayor o igual a 1).

In [5]:
import random

class RingSetup:
    def __init__(self, generator, subgroup_order, n_participants, group):
        # Is the order of the generator correct? For this we check that
        # 1. The subgroup order is prime
        if not is_probably_prime(subgroup_order):
            raise Exception(f"Subgroup_order={subgroup_order} is not a prime number")
        # 2. The generator to the power of subgroup_order is the identity
        if not (generator ** subgroup_order) == group.get_identity():
            raise Exception(f"Generator to the power of subgroup_order is NOT identity")
        # 3. The generator is not the identity
        if generator == group.get_identity():
            raise Exception(f"The generator is the identity")
        ##### completado
        
    
        # Generate a group of participants
        self.participants = [
            Signer(generator, subgroup_order) for _ in range(n_participants)
        ]

        # Store their public keys
        self.public_keys = [x.get_public_key() for x in self.participants]

    def get_public_keys(self):
        return self.public_keys

    def get_random_participant(self):
        return random.choice(self.participants)

Ahora definimos la clase `Signer`.

In [6]:
from hashlib import sha256

class Signer():
    def __init__(self, generator, subgroup_order):
        self.generator = generator
        self.subgroup_order = subgroup_order
        
        # Create and store a secret/public key pair
        # Numero x_A entre {1,....,subgroup_order-1}
        self.secret_key = random.randint(1, self.subgroup_order - 1)
        # Numero y_A = g^x_A
        self.public_key = self.generator ** self.secret_key
        ##### completado

    def get_public_key(self):
        return self.public_key

    # Compute a ring signature for a message and a list of public keys
    def generate_ring_signature(self, message, public_keys):

        # Guardemos el largo.
        n = len(public_keys)
        # Definamos el indice de "partida".
        my_index = public_keys.index(self.public_key)
        next_idx = (my_index + 1) % n
        # Definamos q y g para que se vea como en el enunciado.
        q = self.subgroup_order
        g = self.generator
        # Definamos lista de s_values y c_values, con valores 0 iniciales.
        s_values = [0]*n
        c_values = [0]*n
        # a) Generar r_own propio y calcular c_next = c_{i+1} = h(g^{r_i} || m)
        # Es decir, firmante inicial.
        r_own = random.randint(1, q - 1)
        c_values[next_idx] = int.from_bytes((sha256((str(g ** r_own) + message).encode())).digest(), byteorder='big') % q# Arreglar con % si falla.
        
        # Seguir la cadena en el for.
        for i in range(n-1):
            current_idx = (my_index + 1 + i) % n
            next_idx = (current_idx + 1) % n

            # Generar s_i falso.
            s_i = random.randint(1, q - 1)
            s_values[current_idx] = s_i

            # Calcular g^{r_i} y luego c_{i+1}
            y_i = public_keys[current_idx]
            c_i = c_values[current_idx]

            g_ri = (g ** s_i) * (y_i ** (q - c_i))
            c_values[next_idx] = int.from_bytes((sha256((str(g_ri) + message).encode())).digest(), byteorder='big') % q # Arreglar con % si falla.

        # Cerramos el ciclo.
        c_own = c_values[my_index] # Primer index es el mas importante.
        s_own = (r_own + c_own * self.secret_key) % q
        s_values[my_index] = s_own

        # El enunciado pide devolver (s1, s2, s3, ci, i) donde i es aleatorio.
        challenge_index = random.randint(0, n - 1)
        challenge = c_values[challenge_index]

        return s_values, challenge, challenge_index
            
        ##### completado

**Importante:** En la generación de firmas de anillo se debe calcular el hash de un elemento del grupo concatenado con el mensaje. En esta concatenación puede suponer que la forma de transformar los elementos del grupo a string viene dada por `str(n)`. Además, el resultado del hash se debe interpretar como un número, para lo cual debe usar `int.from_bytes(hash.digest())`.

Ahora vamos con la implementación de una clase para verificar las firmas de anillo, llamada `Verifier`:

In [7]:
class Verifier:
    def __init__(self, generator, subgroup_order):
        self.generator = generator
        self.subgroup_order = subgroup_order

    def verify_ring_signature(self, public_keys, message, signatures, challenge, challenge_index):
        # Verify a ring signature
        # Hacemos igual que antes.
        n = len(public_keys)
        q = self.subgroup_order
        g = self.generator

        # Y definimos el actual:
        current_c = challenge

        # Iteremos, muy similar a antes, pero esta vez iteramos por TODA la lista, y no n-1 (pues antes considerabamos 'next_idx')
        for i in range(n):
            current_idx = (challenge_index + i) % n
            
            s_i = signatures[current_idx]
            y_i = public_keys[current_idx]
            
            g_ri = (g ** s_i) * (y_i ** (q - current_c))

            # Calculamos el proximo c.
            current_c = int.from_bytes((sha256((str(g_ri) + message).encode())).digest(), byteorder='big') % q # Arreglar con % si falla.
            
        # Piden retornar true o false
        return current_c == challenge
        ##### completado

Recuerde que estas clases están definidas para cualquier grupo, y por lo tanto se espera que su implementación funcione con una interfaz genérica para interactuar con estos objetos. Por ejemplo, en el siguiente código se utiliza el grupo Z<sub>643</sub><sup>\*</sup>.

In [8]:
if __name__ == "__main__":
    p, generator, order = 643, 4, 107

    group = ZpStar(p)
    g = group.get_element(generator)

    ring = RingSetup(g, order, 5, group)
    signer = ring.get_random_participant()

    message = "Hello from IIC3253"
    print(f"Message:               {message}")
    print(f"Public keys:           {[str(x) for x in ring.get_public_keys()]}")

    signatures, challenge, challenge_index = signer.generate_ring_signature(
        message, ring.get_public_keys()
    )
    print(f"Signatures:            {signatures}")
    print(f"Challenge index:       {challenge_index}")
    print(f"Challenge:             {challenge}")
    
    verifier = Verifier(g, order)
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Correct signature:     {result}")

    signatures[0] = (signatures[0] - 1) % order
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Incorrect signature:   {result}")

Message:               Hello from IIC3253
Public keys:           ['16', '40', '623', '595', '100']
Signatures:            [86, 24, 90, 103, 50]
Challenge index:       1
Challenge:             44
Correct signature:     True
Incorrect signature:   False


Una vez que haya completado las definiciones de las clases para generar y verificar firmas de anillo, el código anterior debe mostrar algo como lo siguiente:
```
Message:               Hello from IIC3253
Public keys:           ['516', '134', '616', '135', '375']
Signatures:            [86, 105, 97, 77, 63]
Challenge index:       2
Challenge:             27
Correct signature:     True
Incorrect signature:   False
``` 

Como un segundo ejemplo considere un grupo Z<sub>p</sub><sup>\*</sup> que es usado en la práctica.

In [9]:
if __name__ == "__main__":
    p = 17125458317614137930196041979257577826408832324037508573393292981642667139747621778802438775238728592968344613589379932348475613503476932163166973813218698343816463289144185362912602522540494983090531497232965829536524507269848825658311420299335922295709743267508322525966773950394919257576842038771632742044142471053509850123605883815857162666917775193496157372656195558305727009891276006514000409365877218171388319923896309377791762590614311849642961380224851940460421710449368927252974870395873936387909672274883295377481008150475878590270591798350563488168080923804611822387520198054002990623911454389104774092183
    generator = 8041367327046189302693984665026706374844608289874374425728797669509435881459140662650215832833471328470334064628508692231999401840332046192569287351991689963279656892562484773278584208040987631569628520464069532361274047374444344996651832979378318849943741662110395995778429270819222431610927356005913836932462099770076239554042855287138026806960470277326229482818003962004453764400995790974042663675692120758726145869061236443893509136147942414445551848162391468541444355707785697825741856849161233887307017428371823608125699892904960841221593344499088996021883972185241854777608212592397013510086894908468466292313
    order = 63762351364972653564641699529205510489263266834182771617563631363277932854227

    group = ZpStar(p)
    g = group.get_element(generator)

    ring = RingSetup(g, order, 5, group)
    signer = ring.get_random_participant()

    message = "Hello from IIC3253"
    print(f"Message:             {message}")
    print(f"Public keys:         {[str(x) for x in ring.get_public_keys()]}")

    signatures, challenge, challenge_index = signer.generate_ring_signature(
        message, ring.get_public_keys()
    )
    print(f"Signatures:          {signatures}")
    print(f"Challenge index:     {challenge_index}")
    print(f"Challenge:           {challenge}")
    
    verifier = Verifier(g, order)
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Correct signature:   {result}")

    signatures[0] = (signatures[0] - 1) % order
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Incorrect signature: {result}")

Message:             Hello from IIC3253
Public keys:         ['4367852706781483168547869244670958037827428718782964134341288816508809891248935633000849877125399543572181045493350581281842355383255067588593026284882778212700335575290769742054889251740473066586772893262881078964256413280265884214652527633774625407283189990487118338025578473133022862523957364883027895368712914372757775844731209453275664076956909777371407272851936779225410716996435903607390757550524760039998309583928723315893944408581745562753890920183231661802453261543617041940543217532735362030818515992615745816347975433548629100200249332402590761264371995606617054919848614468342514918128353436826454352627', '12594596688999851722728065661477726850077112140364887151955733343341584819939325120020942449215768841257527439218693525047207100787813865426722307136337626900042297267886134017199846018928607272156807333608577276633660241617234046607489596777979857781024848253693150734618409661478056135885128606086810151771430402302

**El código anterior debe mostrar algo como lo siguiente:**
```
Message:             Hello from IIC3253
Public keys:         ['2517830512706408905921795855478002593728536695564369541215433752818688200930590557321299749909051472598879120669335098679086201990588247107010097789653908195551279752437632812718275665598981544118144222483417971539126504478450162602157945937397010648489810647065718469382490168669506251046596417881288288171215855554980819427900171741067720532072516680911006949212485467714683404806945579430182054185594443083586850777040120636465968567638599770255939529206928681124967241910901964546789495870764845450024463889404074510190345750685029474604222569514695003846590369682128367051616414901526467695451326379530045592558', '5270207660733698677379444365513408945379377744136899605872564684007387713297275698259389271331930313222248833279818234133325676502997718277365522335802337127213514973087876911959363974028273837680146332749956406312939587316639326361234287134092889550128946946101251194071838292511417343437360053204333113338720434879059546692509087163054679902770733671764169657087871075083734807460448525636150851282642301952677377786268613136393396241253172078947686682915121559772901880565429197026581606172653634629737742972730555848587200334173694239746043842512888925331708616088881856959738377640477242461071750154708445903616', '4290295718095021988675445254121177064863324098662129262783066199469507095390516614588988784651151081847021838069891388637529024358958231522272516661319379263876975933462254915722181953593832102307038557688856863115076519944152396276190121166385820722775366668295276897317161080467748260348839347932318372823312916424994870126430320104002626579135764282125149078013816804252753118899663976237356011515563337011529792802079272290889759995602735879894050605524591966120664658302635005498072511909994294820012378422696663138258009667356360478704456532326341271584849827662837920312336384196576119296369494018430685468680', '13792847608938719465184496332605495096772014492434818726820448973297625496110337310281695366283943689427744865104761363751586849817037871594690900944374765560486045299130773160170404295646136697636055430219586235874176864304625904266871799915419701399962491391759960375485495587129510617070673009476085756962339636286982291420431354276948191820174224482149383930112834886359364335440659720974265296031634791799861703217132358202332570036784042806999307622122074702481782599438455896989610919983313634466780919462516927530851986252215197877236167006541209043383326112119567715847311805192263845904388190616238648430734', '10011078702742748907877554549063737917030769568096501612653682793110146578391243951572559566683051226189421993140073553893701244152062899876172823506318029601710934733311666584459325204769544040648426353181104063982839933074363401080841815187865036706004492352679601943018686523833801129336689360092848382674867507625546496061084567233837778157788044099412181417511241329506870819948620917334468651158286352700116572819839763989519373583221395470872779149373633862617623122324677676370003431239489306454790584454370900581651101310475827028596893256364366744902336888869996807856811203508203406395657423171781292944255']
Signatures:          [10722411777402596653275815292584573392618152762220609600942110785187707759044, 37006871206013953327579217019949343495896645068493381253815289376515652692533, 9428846663174133374369642171000979807568209932573864941983380633043202379046, 7034677916731842110320680178801060993252839947995940335124833952064825645650, 55236473929309756355522738164922161011054131084796464363101483140615989833630]
Challenge index:     2
Challenge:           46334889421771799592061293299405443101194458819117162433548846027888672198309
Correct signature:   True
Incorrect signature: False
```

## Utilizando curvas elípticas
En esta parte usaremos grupos definidos por curvas elípticas. En particular, debe completar la siguiente definición de la clase `EllipticCurve` considerando la definición de curvas elípticas dada en la ecuación (9.2) del la sección 9.3.4 del libro:

Jonathan Katz y Yehuda Lindell. Introduction to Modern Cryptography. Chapman and Hall/CRC,
tercera edición, 2021.

In [10]:
class EllipticCurve:
    def __init__(self, A, B, p):
        ##### completado
        if (4 * pow(A, 3, p) + 27 * pow(B, 2, p)) % p == 0:
            raise Exception("Tiene raices repetidas!")
        if not is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        self.A = A
        self.B = B
        self.p = p
    
        class Element:
            def __init__(self, x, y = None):
                ##### completado
                if x != None and y != None and ((x < 1 or x > p-1) or (y < 1 or y > p-1)):
                    raise Exception(f"values are not in the range 1,...,{p-1}")
                self.x = x
                self.y = y
                self.A = A
                self.B = B
                self.p = p

            def __eq__(self, other_element):
                ##### completado
                return self.x == other_element.x and self.y == other_element.y

            def __mul__(self, other_element):
                ##### completado
                # Ahora la operación principal es la SUMA!
                # Y Tenemos diferentes casos segun el libro.
                
                # 1) Identidad.
                if self.x is None:
                    return Element(other_element.x, other_element.y)
                if other_element.x is None:
                    return Element(self.x, self.y)

                # 2) If x1 = x2 but y1 != y2 then P1 = −P2 and so P1 + P2 = O.
                if self.x == other_element.x and self.y != other_element.y:
                    return Element(None, None)
                
                # 3) If x1 != x2, then P1 + P2 = (x3, y3).
                if self.x != other_element.x:
                    inv = pow((other_element.x - self.x) % p, -1, p)
                    s = (((other_element.y - self.y) % p) * inv) % p

                    x_3 = (s * s - self.x - other_element.x) % p
                    y_3 = (s * (self.x - x_3) - self.y) % p
                    return Element(x_3, y_3)

                # 4) If P1 = P2 and y1 != 0 then P1 + P2 = 2P1 = (x3, y3).
                if self.x == other_element.x and self.y == other_element.y and self.y != 0:
                    inv = pow((2 * self.y) % p, -1, p)
                    s = (((3 * self.x * self.x + A) % p) * inv) % p

                    x_3 = (s * s - self.x - other_element.x) % p
                    y_3 = (s * (self.x - x_3) - self.y) % p
                    return Element(x_3, y_3)
                    
                # 5) If P1 = P2 and y1 = 0 then P1 + P2 = 2P1 = O.
                if self.x == other_element.x and self.y == other_element.y and self.y == 0:
                    return Element(None, None)
                
                    
            def __pow__(self, exponent):
                ##### completado
                # Lo de abajo internet lo recomienda como una version optimizada: Double and add.
                # La estrategia es apoyarse en la idea de que cualquier entero se puede escribir en binario así:
                # n = b0 * 2^0 + b1 * 2^1 + b2 * 2^2... etc.
                # Luego entonces es mas rapido hacer 
                result = Element(None, None)  # identidad
                current = self # Punto actual
                while exponent > 0: 
                    if exponent % 2 == 1: # Si el bit menos significativo es 1, sumar.
                        result = result * current # Hacer la suma del result con el current. SOLO SI EL BIT ES 1.
                    current = current * current # Doblar el punto (double). Obvio porque estamos pasando al proximo bit. Ej: de 2^3 a 2^4.
                    # El current anterior entonces es simplemente decir 2*P. Cada vez.
                    exponent //= 2 # Pasar al siguiente bit
                return result

            def __str__(self):
                ##### completado
                if self.x is None:
                    return "O"
                return f"({self.x},{self.y})"
                    
        self.element_class = Element
                
    def get_identity(self):
        ##### completado
        return self.element_class(None, None)
         
    def get_element(self, x, y):
        ##### completado
        return self.element_class(x, y)
        

En esta definición de `EllipticCurve`, dado un número primo `p`, cada punto sobre la curva es un par ordenado `(x,y)` con `x` e `y` en el conjunto `{0, ..., p-1}`, excepto por el neutro del grupo que un elemento especial que no necesita notación de par ordenado (ver el libro de Katz & Lindell para una explicación de esto). Por esto el constructor de la clase `EllipticCurve` recibe dos argumentos para representar un par ordenado, y también considera el caso en que `y` no esté definido porque se está utilizando el elemento neutro.

**Importante:** Dado un elemento `g = (x, y)` del grupo, diremos que su interpretación como string es literalmente `(x, y)` (notar el espacio después de la coma). Es decir, la interpretación como string se debe calcular de la forma `f"({self.x}, {self.y})"`.

De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `EllipticCurve` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `p` no es un número primo, entonces se debe generar una excepción.

Su definición de la clase `EllipticCurve` va a ser utilizada por la implementación del protocolo de firmas de anillos de la misma forma que para la clase `ZpStar`. Por ejemplo, en el siguiente código se utilizan firmas de anillos sobre la curva elíptica [P-256](https://neuromancer.sk/std/nist/P-256).

In [11]:
if __name__ == "__main__":
    p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
    A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
    order = 115792089210356248762697446949407573529996955224135760342422259061068512044369

    group = EllipticCurve(A, B, p)
    g = group.get_element(g_x, g_y)
    
    ring = RingSetup(g, order, 5, group)
    signer = ring.get_random_participant()

    message = "Hello from IIC3253"
    print(f"Message:             {message}")
    print(f"Public keys:         {[str(x) for x in ring.get_public_keys()]}")

    signatures, challenge, challenge_index = signer.generate_ring_signature(
        message, ring.get_public_keys()
    )
    print(f"Signatures:          {signatures}")
    print(f"Challenge index:     {challenge_index}")
    print(f"Challenge:           {challenge}")
    
    verifier = Verifier(g, order)
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Correct signature:   {result}")

    signatures[0] = (signatures[0] - 1) % order
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Incorrect signature: {result}")

Message:             Hello from IIC3253
Public keys:         ['(51505349421046560593890460823383846765818734334128073872908364705662552424909,6553049394526456867760617474270287507627805207795786125947885015357124518743)', '(88170754290547145582453871607443191859029137186954757503649882333385643659213,101041172809142182619596323632248620736988079145956441115013753689378556984536)', '(40850750920455135674677140892952961284817663292964904045257350992340213836869,34059299617129564503066173447821567866528664411953992351860023692439393324274)', '(13122328889820926097288012929024177704754689921320080014598991167774161709259,37137182900615958995129725338800640511469318614603199911509580425974250450623)', '(19181015942825810464438886537878388308193310329568297172723646748566211417781,38024502526633364510824684910411538538761656639882287696110747251687723553500)']
Signatures:          [42137709288101405143704636501413548490670974903734529424081542910844730426467, 95829870031262927321118435313443

**El código anterior debe mostrar algo como lo siguiente:**

Message:             Hello from IIC3253

Public keys:
['(36228047005606623638394851230173192734851072661042561449319473736621754039520, 20172362822856456120421044511946279043257074756688522685143706143736092565958)', '(58564759885583515378683115765772107948142614786701727763264984960945695330334, 79129698021788006657426468620527515930126766311369828883900257058578878052857)', '(25108858240980601702082892074061491934513031462242737558923411453375449067242, 76425488572625390607396261141639007962966296877847096915671747178845593774896)', '(6701657221200507285541433974699587733893239378552437797081211908168708912451, 97013597839749465309655953073668617169117780346910872542402314282358373971520)', '(113015587168257705614560565034856000733681336982911328860838726980840022492723, 54104620305347279303971506213405538556140509029853428898553160506673533048482)']

Signatures:
[77654962933025577729744678855999034023206810390864799379806439528613544048007, 72513278071731766598881593643721383994593537425755886558349450371764918110449, 28944527442594298721839943005624633745105995076212793147845350194880884108800, 90231818693911206835713485736658448057422079278315344453989541044751933614865, 25344903812783089196604799883051329695362840222494910906975057115807269376447]

Challenge index:     2

Challenge:           86130958050425514850634115208684686516639197236630218119818914977367756079082

Correct signature:   True

Incorrect signature: False

## ¡Buena suerte con la tarea!